<a href="https://colab.research.google.com/github/jessiepsx/chatbot_alura/blob/main/An%C3%A1lise_de_Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
%pip -q install google-genai

In [37]:
!pip install -q google-adk

In [ ]:
import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from datetime import date
import warnings
import textwrap
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")

# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


# Função auxiliar para exibir texto formatado
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


# Função para criar e executar o Agente
def criar_agente_verificador_noticias():
    agente_verificador = Agent(
        name="agente_verificador_noticias",
        model="gemini-2.0-flash",  # Usando gemini-pro para melhor raciocínio
        instruction="""
        Você é o Agente Sherlock Holmes, um detetive de notícias. Sua tarefa é analisar manchetes e determinar se elas são verdadeiras ou falsas com um toque de humor e sagacidade.
        Use a ferramenta de busca do Google (google_search) para encontrar pistas e evidências.
        Considere os seguintes aspectos ao analisar a manchete:

        -   **Fonte:** Essa fonte é confiável ou mais suspeita que um gato preto em noite de lua cheia?
        -   **Evidências:** Há alguma prova concreta, ou estamos apenas lidando com fumaça e espelhos?
        -   **Consistência:** Essa história se encaixa no que já sabemos, ou é mais furada que queijo suíço?
        -   **Linguagem:** A manchete grita "clickbait" ou "sensacionalismo"? Se sim, desconfie!

        Classifique a manchete como:
        -   "Possivelmente Verdadeira": Se a manchete vier de uma fonte confiável, for apoiada por evidências sólidas e se encaixar no que já sabemos.
        -   "Suspeita": Se algo cheirar mal, não houver provas ou a manchete parecer mais exagerada que um filme de ação de Hollywood.

        Forneça uma explicação, no estilo de um detetive astuto, citando suas fontes. Seja criativo e engraçado.
        """,
        description="Agente que verifica a veracidade de manchetes de notícias",
        tools=[google_search]
    )

    return agente_verificador



def verificar_manchete(agente, manchete):
    """
    Função para verificar uma manchete usando o agente verificador de notícias.

    Args:
        agente (Agent): O agente verificador de notícias.
        manchete (str): A manchete a ser verificada.

    Returns:
        str: A resposta do agente, classificando a manchete e fornecendo uma explicação.
    """
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agente.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agente, app_name=agente.name, session_service=session_service)
    conteudo = types.Content(role="user", parts=[types.Part(text=manchete)])

    resposta_final = ""
    for evento in runner.run(user_id="user1", session_id="session1", new_message=conteudo):
        if evento.is_final_response():
            for parte in evento.content.parts:
                if parte.text is not None:
                    resposta_final += parte.text + "\n"
    return resposta_final



# Cria o agente
agente_verificador = criar_agente_verificador_noticias()

# Loop para verificar múltiplas manchetes
while True:
    manchete = input("Por favor, digite a manchete que você deseja verificar (ou 'sair' para encerrar): ")
    if manchete.lower() == "sair":
        break  # Encerra o loop se o usuário digitar "sair"

    # Verifica a manchete
    resultado = verificar_manchete(agente_verificador, manchete)

    # Exibe o resultado
    print("\n--- 🕵️‍♂️🔎 Resultado da Verificação da Manchete ---\n")
    display(to_markdown(resultado))
    print("--------------------------------------------------------------")
